In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sqlite3 as sql

from src import _02_convert_sql as convert
from src import _02_create_charts as charts

connection = sql.connect('data/taylor_swift.db')
cursor = connection.cursor()

In [2]:
eras = {
    'Taylor Swift': '#A5C9A5',
    'Fearless (TV)': '#EFC180',
    'Speak Now (TV)': '#C7A8CB',
    'Red (TV)': '#7A2E39',
    '1989 (TV)': '#B5E5F8',
    'reputation': '#746F70',
    'Lover': '#F7B0CC',
    'folklore': '#CDC9C1',
    'evermore': '#C5AC90',
    'Midnights': '#242E47'
}

In [3]:
credit_temp_tables = convert.sql_to_string('02_credit_temp_tables.sql')
cursor.executescript(credit_temp_tables)

In [4]:
pd.read_sql('SELECT * FROM writer_count_era LIMIT 5', connection)

,era,writer,writer_count,total_songs_writer
0,Midnights,Aaron Dessner,4,27
1,evermore,Aaron Dessner,14,27
2,folklore,Aaron Dessner,9,27
3,1989 (TV),Ali Payami,1,2
4,reputation,Ali Payami,1,2


In [5]:
pd.read_sql('SELECT * FROM producer_count_era LIMIT 5', connection)

,era,producer,producer_count,total_songs_producer
0,Fearless (TV),Aaron Dessner,2,41
1,Midnights,Aaron Dessner,3,41
2,Red (TV),Aaron Dessner,4,41
3,Speak Now (TV),Aaron Dessner,3,41
4,evermore,Aaron Dessner,17,41


In [6]:
pd.read_sql('SELECT * FROM artist_count_era LIMIT 5', connection)

,era,artist,artist_count,total_songs_artist
0,Fearless (TV),BOYS LIKE GIRLS,1,1
1,evermore,Big Red Machine,1,1
2,evermore,Bon Iver,1,2
3,folklore,Bon Iver,1,2
4,Lover,Brendon Urie,1,1


In [7]:
collaborator_temp_table = convert.sql_to_string('02_collaborator_temp_table.sql')
cursor.executescript(collaborator_temp_table)

In [8]:
pd.read_sql('SELECT * FROM collaborator_flat LIMIT 5', connection)

,era,collaborator,writer_count,producer_count,artist_count,total_songs_writer,total_songs_producer,total_songs_artist,total_credits
0,1989 (TV),Ali Payami,1,0,0,2,0,0,2
1,1989 (TV),Calvin Harris,1,1,1,1,1,1,3
2,1989 (TV),Christopher Rowe,0,10,0,0,61,0,61
3,1989 (TV),Diane Warren,1,0,0,1,0,0,1
4,1989 (TV),Imogen Heap,1,1,0,1,1,0,2


In [9]:
collaborators_query = convert.sql_to_string('02_collaborators_final.sql')
collaborators = pd.read_sql(collaborators_query, connection)

# Set collaborator order to have 'One-Time Collaborators' appear last
collaborator_order = sorted(set(collaborators['collaborator']) - {'One-Time Collaborators'}) + ['One-Time Collaborators']
collaborators['collaborator'] = pd.Categorical(collaborators['collaborator'], categories=collaborator_order, ordered=True)

# Set era order to be chronological
collaborators['era'] = pd.Categorical(collaborators['era'], eras.keys())
collaborators.sort_values('era', inplace=True)
collaborators.head(10)

,era,collaborator,writer_count,producer_count,artist_count
68,Taylor Swift,Troy Verges,1,1,0
67,Taylor Swift,Robert Ellis Orrall,3,5,0
66,Taylor Swift,One-Time Collaborators,1,0,0
65,Taylor Swift,Nathan Chapman,1,12,0
64,Taylor Swift,Liz Rose,8,0,0
63,Taylor Swift,Brett James,1,1,0
62,Taylor Swift,Angelo Petraglia,2,1,0
18,Fearless (TV),Liz Rose,8,0,0
23,Fearless (TV),Tommy Lee James,2,0,0
22,Fearless (TV),Robert Ellis Orrall,1,0,0


In [10]:
writers_chart = stacked_bar_chart('barh', writers, 'writer', 'song_count', 'era', eras, 'Collaborators by Era - Writers', 'Total Writing Credits', 'Writer', 'Eras')

NameError: name 'stacked_bar_chart' is not defined

In [ ]:
producers_chart = stacked_bar_chart('barh', producers, 'producer', 'song_count', 'era', eras, 'Collaborators by Era - Producers', 'Total Producing Credits', 'Producer', 'Eras')

In [ ]:
connection.close()